<a href="https://colab.research.google.com/github/ImpulsoGov/mensageria-mvp/blob/main/mvp_algoritmo_divisao_equipes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **Configuração para divisão dos grupos de teste e controle dos municípios**

##### Configurações iniciais do ambiente

In [1]:
# autenticação
from google.colab import auth
auth.authenticate_user()

In [2]:
# conexão BQ
from google.cloud import bigquery
client = bigquery.Client(project='predictive-keep-314223')

In [3]:
# importação de bibliotecas
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from google.oauth2 import service_account

##### Consulta aos dados

In [4]:
query = """
    SELECT *
    FROM `predictive-keep-314223.ip_mensageria_camada_prata.unificado_lista_com_telefones_grupos_atendimentos`
"""

query_job = client.query(query)
df = query_job.to_dataframe()

##### Preparação dos dados

In [5]:
# exclusão de cidadãos com registro de falecimento
df = df[df['se_faleceu']==0]

In [6]:
# identificação dos cidadãos com pendências em cada linha de cuidado
df['citopatologico_pendente'] = df['status_exame'].apply(lambda x: True if x in ('exame_nunca_realizado','exame_vencido','exame_vence_no_quadrimestre_atual') else False)

<ipython-input-6-b78de1522804>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['citopatologico_pendente'] = df['status_exame'].apply(lambda x: True if x in ('exame_nunca_realizado','exame_vencido','exame_vence_no_quadrimestre_atual') else False)


In [7]:
# função para identificar hipertensos em pendência
def pendencia_hipertensos(x):
    if x['esta_na_lista_hipertensos'] == True and (x['realizou_afericao_ultimos_6_meses'] == False or x['realizou_consulta_hip_ultimos_6_meses'] == False):
        return True
    else:
        return False

df['hipertensao_pendente'] = df.apply(pendencia_hipertensos,axis=1)

In [8]:
# função para identificar diabeticos em pendência
def pendencia_diabeticos(x):
    if x['esta_na_lista_diabeticos'] == True and (x['realizou_consulta_dia_ultimos_6_meses'] == False or x['realizou_solicitacao_hemoglobina_ultimos_6_meses'] == False):
        return True
    else:
        return False

df['diabetes_pendente'] = df.apply(pendencia_diabeticos,axis=1)

In [9]:
#calcular preenchimento de telefones
def calcula_percentual(df,telefone,esta_na_lista_citopatologico,esta_na_lista_hipertensos,esta_na_lista_diabeticos,municipio,equipe_ine,equipe_nome,):
    df_validos = df[df[telefone].notna() & (df[telefone] != '')]
    agrupado = df_validos.groupby([municipio,equipe_ine,equipe_nome,esta_na_lista_citopatologico,esta_na_lista_hipertensos,esta_na_lista_diabeticos])[telefone].nunique().reset_index(name='Telefones únicos')
    total_cuidado = df.groupby([municipio,equipe_ine,equipe_nome,esta_na_lista_citopatologico,esta_na_lista_hipertensos,esta_na_lista_diabeticos]).size().reset_index(name='Total cidadãos')
    resultado=pd.merge(agrupado, total_cuidado, on=[municipio,equipe_ine,equipe_nome,esta_na_lista_citopatologico,esta_na_lista_hipertensos,esta_na_lista_diabeticos], how='right')
    resultado['Percentual preenchido'] = (resultado['Telefones únicos']/resultado['Total cidadãos'])*100

    resultado['Telefones únicos'].fillna(0,inplace=True)
    resultado['Percentual preenchido'].fillna(0,inplace=True)

    return resultado

resultado=calcula_percentual(df,'celular_tratado','esta_na_lista_citopatologico','esta_na_lista_hipertensos','esta_na_lista_diabeticos','municipio','equipe_ine','equipe_nome')
resultado


<ipython-input-9-fc300ab3454a>:9: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  resultado['Telefones únicos'].fillna(0,inplace=True)
<ipython-input-9-fc300ab3454a>:10: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try u

,municipio,equipe_ine,equipe_nome,esta_na_lista_citopatologico,esta_na_lista_hipertensos,esta_na_lista_diabeticos,Telefones únicos,Total cidadãos,Percentual preenchido
0,Alagoinha,0000134945,PSF 03 - FREI ESTEVAO,False,False,True,3,4,75.000000
1,Alagoinha,0000134945,PSF 03 - FREI ESTEVAO,False,True,False,40,52,76.923077
2,Alagoinha,0000134945,PSF 03 - FREI ESTEVAO,False,True,True,23,35,65.714286
3,Alagoinha,0000134945,PSF 03 - FREI ESTEVAO,True,False,False,43,53,81.132075
4,Alagoinha,0000134945,PSF 03 - FREI ESTEVAO,True,False,True,3,5,60.000000
...,...,...,...,...,...,...,...,...,...
761,Vitorino Freire,0002231484,ESF OZEAS CASTRO II,False,True,True,13,24,54.166667
762,Vitorino Freire,0002231484,ESF OZEAS CASTRO II,True,False,False,29,46,63.043478
763,Vitorino Freire,0002231484,ESF OZEAS CASTRO II,True,False,True,1,1,100.000000
764,Vitorino Freire,0002231484,ESF OZEAS CASTRO II,True,True,False,10,18,55.555556


In [10]:
from google.colab import drive
drive.mount ('/drive')
resultado.to_csv('/drive/MyDrive/preenchimento_telefones.csv', index=False)

Mounted at /drive


In [11]:
#seleção de cidadãos com pendências
df_aptos_mensagens = df[(df['diabetes_pendente'] == True) | (df['hipertensao_pendente'] == True) | (df['citopatologico_pendente'] == True)]

#### Priorização das linhas de cuidado

In [12]:
# função para identificar cronicos em pendência
def pendencia_cronicos(x):
    if x['hipertensao_pendente'] or x['diabetes_pendente']:
        return True
    else:
        return False

df_aptos_mensagens['cronicos_pendente'] = df_aptos_mensagens.apply(pendencia_cronicos,axis=1)

<ipython-input-12-e13c521f5830>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_aptos_mensagens['cronicos_pendente'] = df_aptos_mensagens.apply(pendencia_cronicos,axis=1)


In [13]:
# função para priorização da linha de cuidado -- caso seja parte de duas linhas de cuidado dividir aleatoriamente e igualmente entre crônicos ou cito

def linha_cuidado(x, y):
    if x and not y:
        return 'citopatologico'
    elif y and not x:
        return 'cronicos'
    else:
        return np.nan

df_aptos_mensagens['linha_cuidado'] = df_aptos_mensagens.apply(lambda row: linha_cuidado(row['citopatologico_pendente'], row['cronicos_pendente']), axis=1)

interseccao = df_aptos_mensagens['linha_cuidado'].isna()
total = interseccao.sum()

atribuicao = ['citopatologico'] * (total // 2) + ['cronicos'] * (total - total // 2)
np.random.shuffle(atribuicao)

df_aptos_mensagens.loc[interseccao, 'linha_cuidado'] = atribuicao


<ipython-input-13-b77bb472daa3>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_aptos_mensagens['linha_cuidado'] = df_aptos_mensagens.apply(lambda row: linha_cuidado(row['citopatologico_pendente'], row['cronicos_pendente']), axis=1)


In [14]:
df_aptos_mensagens

,municipio,nome_do_paciente,celular_tratado,cns,cpf,data_de_nascimento,ds_dim_situacao_trabalho,ds_identidade_genero,no_social_cidadao,numero_visitas_ubs_ultimos_12_meses,...,possui_diabetes_autorreferida,possui_diabetes_diagnosticada,realizou_consulta_dia_ultimos_6_meses,realizou_solicitacao_hemoglobina_ultimos_6_meses,mensagem_data_envio,citopatologico_pendente,hipertensao_pendente,diabetes_pendente,cronicos_pendente,linha_cuidado
4,Marajá do Sena,RAIMUNDO DIAS,9982755267,<NA>,94979200320,1975-04-03,Outro,Não informado,None,0,...,False,False,False,False,,False,True,False,True,cronicos
9,Marajá do Sena,VALMIR GOMES SILVA,9985006707,898003404845244,94599670359,1954-10-15,Aposentado / Pensionista,Não informado,None,0,...,False,False,False,False,,False,True,False,True,cronicos
11,Marajá do Sena,GARDENIA SILVA LOURENCO,0,<NA>,61125298316,1996-02-12,Não informado,Não informado,None,0,...,False,False,False,False,,True,False,False,False,citopatologico
18,Marajá do Sena,IRANEIDE SANTOS DA SILVA CONCEICAO,9898476793,<NA>,97693502300,1977-05-30,Não informado,Não informado,None,0,...,False,False,False,False,,True,False,False,False,citopatologico
24,Marajá do Sena,JORDAO ALEXANDRE DE OLIVEIRA,98985051891,<NA>,98706403387,1956-02-28,Não informado,Não informado,None,1,...,False,False,False,False,,False,True,False,True,cronicos
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
147567,Monsenhor Tabosa,ZILMAR PEREIRA DOS SANTOS,8836962175,<NA>,83122656353,1976-01-28,Autônomo sem previdência social,Não informado,None,0,...,False,False,False,False,,True,False,False,False,citopatologico
147571,Monsenhor Tabosa,ZULEIDE FERREIRA DE BARROS,8836962170,702402537247520,89691610310,1977-11-11,Não informado,Não informado,None,0,...,False,False,False,False,,True,False,False,False,citopatologico
147572,Monsenhor Tabosa,ZULEIDE SOUZA DA LUZ,8894038721,705001080632757,97348546315,1972-12-01,Desempregado,Não informado,None,4,...,False,False,False,False,,True,True,False,True,cronicos
147573,Monsenhor Tabosa,ZULEIDE SOUZA DA LUZ,8894038721,705001080632757,97348546315,1972-12-01,Outro,Não informado,None,4,...,False,False,False,False,,True,True,False,True,cronicos


In [15]:
df_equipes = pd.DataFrame(df_aptos_mensagens.pivot_table(index=['municipio','equipe_ine','equipe_nome'], columns='linha_cuidado', values='nome_do_paciente',aggfunc='count', fill_value=0))
df_equipes.reset_index(inplace=True)
# exclusão de registros sem equipe atribuída
df_equipes = df_equipes[df_equipes['equipe_ine']!='0']
df_equipes

linha_cuidado,municipio,equipe_ine,equipe_nome,citopatologico,cronicos
0,Alagoinha,0000134945,PSF 03 - FREI ESTEVAO,64,106
1,Alagoinha,0000134953,USF 04 DR ORISVALDO I DA SILVA,53,200
2,Alagoinha,0000134961,PSF 02,18,49
3,Alagoinha,0000134988,PSF 01 MANOEL BEZERRA DA SILVA,135,207
4,Alagoinha,0000134996,PSF 05 - BOM SUCESSO,21,58
...,...,...,...,...,...
114,Vitorino Freire,0000064343,ESF BREJO DAS FLORES,48,55
115,Vitorino Freire,0000064351,ESF RENASCENCA,250,263
116,Vitorino Freire,0002094657,MUTIRAO,95,55
117,Vitorino Freire,0002094665,OLHO D AGUA MANOEL LUIS,44,55


#### Randomização e divisão dos grupos

In [16]:
# randomização os grupos de teste e controle por equipes
controle, teste = train_test_split(df_equipes, test_size=0.5, random_state=42,stratify=df_equipes['municipio'])

controle['grupo'] = 'controle'
teste['grupo'] = 'teste'
df_equipes = pd.concat([controle, teste])
df_equipes


linha_cuidado,municipio,equipe_ine,equipe_nome,citopatologico,cronicos,grupo
26,Brejo de Areia,0001925806,EQUIPE BREJO NOVO,101,73,controle
82,Paulo Ramos,0000054755,ESF VILA RAPINA - I,130,145,controle
104,Vitorino Freire,0000064246,ESF JACARE,120,83,controle
7,Baraúna,0000107263,ESF VI,41,46,controle
13,Baraúna,0000107344,ESF X,22,42,controle
...,...,...,...,...,...,...
118,Vitorino Freire,0002231484,ESF OZEAS CASTRO II,60,44,teste
116,Vitorino Freire,0002094657,MUTIRAO,95,55,teste
99,Tarrafas,0000104825,EQUIPE III,420,644,teste
85,Paulo Ramos,0002214261,ESF CENTRO DOS LEITE - II,33,29,teste


In [17]:
df_grupos = df_aptos_mensagens.merge(df_equipes[['equipe_ine','grupo']], how='left',on='equipe_ine')

In [18]:
df_grupos.drop_duplicates(inplace=True)
df_grupos = df_grupos[df_grupos['equipe_ine']!='0']

#### Salvando os dados de divisão de grupos numa tabela do BQ para seleção diária

In [19]:
df_grupos.info()

<class 'pandas.core.frame.DataFrame'>
Index: 23240 entries, 0 to 23270
Data columns (total 39 columns):
 #   Column                                            Non-Null Count  Dtype  
---  ------                                            --------------  -----  
 0   municipio                                         23240 non-null  object 
 1   nome_do_paciente                                  23240 non-null  object 
 2   celular_tratado                                   23200 non-null  Int64  
 3   cns                                               14469 non-null  Int64  
 4   cpf                                               23240 non-null  Int64  
 5   data_de_nascimento                                23240 non-null  dbdate 
 6   ds_dim_situacao_trabalho                          23240 non-null  object 
 7   ds_identidade_genero                              23240 non-null  object 
 8   no_social_cidadao                                 1909 non-null   object 
 9   numero_visitas_ubs_ult

In [20]:
# criação da tabela para guardar a análise inicial dos cidadãos
table_id = "predictive-keep-314223.ip_mensageria_camada_prata.divisao_teste_controle_equipes"
job = client.load_table_from_dataframe(df_grupos, table_id)
job.result()

LoadJob<project=predictive-keep-314223, location=US, id=c65e4440-d6f6-4e87-b5e8-3956f3258e9b>

#### Verificação de preenchimento de novos campos

In [21]:
df_grupos.ds_dim_situacao_trabalho.value_counts()

,count
ds_dim_situacao_trabalho,
Não informado,7158
Aposentado / Pensionista,6088
Outro,3389
Não trabalha,1878
Desempregado,1513
Autônomo sem previdência social,1017
Assalariado com carteira de trabalho,667
Assalariado sem carteira de trabalho,626
Servidor público / Militar,508


In [22]:
df_grupos.st_plano_saude_privado.value_counts()

,count
st_plano_saude_privado,
0,23047
1,193


In [23]:
df_grupos.st_comunidade_tradicional.value_counts()

,count
st_comunidade_tradicional,
0,22720
1,520


In [24]:
df_grupos.ds_identidade_genero.value_counts()

,count
ds_identidade_genero,
Não informado,22838
Mulher cisgênero,174
Outro,86
Homem cisgênero,64
Homem transgênero,46
Mulher transgênero,31
Transgênero,1


#### Criação da estrutura da tabela para guardar o histórico de envios

In [29]:
df_historico_envios = pd.DataFrame(columns=['municipio', 'equipe_ine', 'equipe_nome', 'linha_cuidado','nome_do_paciente','data_de_nascimento','celular_tratado','horario_grupo','data_envio','grupo'])
df_historico_envios = df_historico_envios.astype({'data_de_nascimento': 'datetime64[ns]', 'data_envio': 'datetime64[ns]','municipio': 'string','equipe_ine':'string', 'equipe_nome':'string', 'linha_cuidado':'string','nome_do_paciente':'string','celular_tratado':'string','horario_grupo':'string','grupo':'string'})
df_historico_envios

,municipio,equipe_ine,equipe_nome,linha_cuidado,nome_do_paciente,data_de_nascimento,celular_tratado,horario_grupo,data_envio,grupo


In [30]:
# criação da tabela para guardar o histórico de envios
table_id = "predictive-keep-314223.ip_mensageria_camada_prata.historico_envio_mensagens"
job = client.load_table_from_dataframe(df_historico_envios, table_id)
job.result()

LoadJob<project=predictive-keep-314223, location=US, id=1f2f7d68-3218-4f63-a279-93fdbb68886a>